In [1]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import functools
from sklearn.metrics import accuracy_score 
import gensim

In [2]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 6.4 MB/s eta 0:00:00


READING THE DATA

In [5]:
import pandas as pd

df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)
df_test = pd.read_csv('/content/merged_dataset.csv')
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_test.head()

<ipython-input-5-15018a8d7b55>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)
<ipython-input-5-15018a8d7b55>:3: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)


,content
0,"Don’t worry, it’s just a prop"
1,It never should have happened but that’s exac...
2,"Like really, it’s the depressed kids you’re go..."
3,"Hey there, if you're feeling a bit lost and ov..."
4,no. they are meant to spur demand after elon...


In [6]:
df

,clean_text,is_depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1
...,...,...
7726,is that snow,0
7727,moulin rouge mad me cry once again,0
7728,trying to shout but can t find people on the list,0
7729,ughh can t find my red sox hat got ta wear thi...,0


DATA CLEANING AND PREPROCESSING

In [7]:
df['clean_text'] = df['clean_text'].str.replace(r'<[^<>]*>', '', regex=True)#1
df['clean_text'] = df['clean_text'].str.replace(r'http://\S+|https://\S+', '', regex=True)#2
df['clean_text']=df['clean_text'].str.lower()
df['clean_text'] = df['clean_text'].str.replace(r'[@_!#$%^&*()<>?/\|}{~:]', '', regex=True)#5
df['clean_text'] = df['clean_text'].replace("\s+", " ", regex=True).str.strip()#last
after_dc=df['clean_text'].str.len().mean()
df_test['clean_text'] = df_test['content'].str.replace(r'<[^<>]*>', '', regex=True)#1
df_test['clean_text'] = df_test['clean_text'].str.replace(r'http://\S+|https://\S+', '', regex=True)#2
df_test['clean_text']=df_test['clean_text'].str.lower()
df_test['clean_text'] = df_test['clean_text'].str.replace(r'[@_!#$%^&*()<>?/\|}{~:]', '', regex=True)#5
df_test['clean_text'] = df_test['clean_text'].replace("\s+", " ", regex=True).str.strip()#last
#after_dc=df['clean_text'].str.len().mean()

#DATA PREPOCESSING
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')


def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)



# Lemmatizing
df['clean_text'] = df['clean_text'].apply(lambda x: lemmatize_sentence(x))
df_test['clean_text'] = df_test['clean_text'].apply(lambda x: lemmatize_sentence(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [8]:
df_test = df_test.drop(['content'], axis = 1)
df_test.head()

,clean_text
0,"don ’ t worry , it ’ s just a prop"
1,it never should have happen but that ’ s exact...
2,"like really , it ’ s the depressed kid you ’ r..."
3,"hey there , if you 're feel a bit lose and ove..."
4,no . they be mean to spur demand after elon bo...


WORD2VEC MODEL

In [9]:
#The pretrained googleW2V model is loaded to generate the word embeddings
import gensim.downloader as googleW2V
google_w2v = googleW2V.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


SPLITTING DATA INTO TRAIN AND TEST

In [10]:
X=df['clean_text']
y=df['is_depression']

X_unlabeled = df_test['clean_text']
X_unlabeled.head()

0                   don ’ t worry , it ’ s just a prop
1    it never should have happen but that ’ s exact...
2    like really , it ’ s the depressed kid you ’ r...
3    hey there , if you 're feel a bit lose and ove...
4    no . they be mean to spur demand after elon bo...
Name: clean_text, dtype: object

In [11]:
#dataset is split into training and testing sets and the indexes are reset 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [12]:
X_train = X_train.reset_index(drop=True)

In [13]:
X_test = X_test.reset_index(drop=True)

In [14]:
y_train = y_train.reset_index(drop=True)

In [15]:
y_test = y_test.reset_index(drop=True)

AVERAGE OF THE VECTORS

In [16]:
#The average is found between all the vectors corresponding to the words in a sentence.
#Average found and appended for each sentence for the training set
X_train1 = []
X_unlabeled1 = []

for x in X_train:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = google_w2v[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_train1.append(np.mean(wordveclist, axis=0))

for x in X_unlabeled:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = google_w2v[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_unlabeled1.append(np.mean(wordveclist, axis=0)) 

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
#Average found and appended for each sentence for the test set
X_test1 = []
for x in X_test:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = google_w2v[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_test1.append(np.mean(wordveclist, axis=0)) 

In [18]:
X_unlabeled1[:5]

[array([-2.40652896e-02,  7.53672495e-02, -8.52399543e-02,  1.20186940e-01,
        -9.16748047e-02,  1.88337057e-03,  2.46276855e-02, -6.94754496e-02,
         1.25313893e-01, -7.44628906e-03, -5.99670410e-03, -1.12409316e-01,
        -6.56999871e-02,  2.71944311e-02, -1.67445585e-01,  1.47251680e-01,
         9.30960551e-02,  1.55587330e-01, -4.85316701e-02, -1.27790183e-01,
        -9.62960348e-02,  3.05873323e-02,  1.12670898e-01, -1.95922852e-02,
         8.67396742e-02,  4.01349738e-02, -7.62416273e-02, -4.43551205e-02,
         2.16500424e-02, -4.72935252e-02, -1.13568986e-02,  1.09165736e-01,
        -8.04530531e-02,  2.85382960e-02,  1.74386165e-04,  4.23060842e-02,
        -5.72596975e-02,  1.14593506e-01,  2.10658479e-02,  1.45019531e-01,
         1.32108415e-02, -1.01087295e-01,  1.89383373e-01, -4.29240651e-02,
        -8.00868403e-03,  2.82854345e-02, -7.32596293e-02, -9.91249084e-03,
         2.83028744e-02,  1.35881692e-01, -7.29806051e-02,  1.34717673e-01,
         4.1

In [19]:
#non word vector values are removed 
wv_train = []
for i, x in enumerate(X_train1):
    try:
        len(x)
        wv_train.append(i)
    except:
        print(i)
        print(x)
        
wv_unlabeled = []
for i, x in enumerate(X_unlabeled1):
    try:
        len(x)
        wv_unlabeled.append(i)
    except:
        print(i)
        print(x)


wv_test = []
for i, x in enumerate(X_test1):
    try:
        len(x)
        wv_test.append(i)
    except:
        print(x)
X_train1 = [X_train1[i] for i in wv_train]
X_test1 = [X_test1[i] for i in wv_test]
X_unlabeled1 = [X_unlabeled1[i] for i in wv_unlabeled]
y_train1 = [y_train[i] for i in wv_train]
y_test1 = [y_test[i] for i in wv_test]

2088
nan
2420
nan
2961
nan
3536
nan
3798
nan
4738
nan
4739
nan
4985
nan
5114
nan
5123
nan
5511
nan
5513
nan
5972
nan
32
nan
3142
nan
3163
nan
3584
nan
3832
nan
5356
nan
6481
nan
8596
nan
8639
nan
9802
nan
9907
nan
11492
nan
11502
nan
11876
nan
12570
nan
13443
nan
13644
nan
14035
nan
14200
nan
14769
nan
15572
nan
16747
nan
18189
nan
18430
nan
18431
nan
18913
nan
18920
nan
18922
nan
18929
nan
18940
nan
18976
nan
19018
nan
19020
nan
19096
nan
19120
nan
19194
nan
19209
nan
19288
nan
19387
nan
19526
nan
19534
nan
19539
nan
19573
nan
19633
nan
19642
nan
19657
nan
19671
nan
19714
nan
19756
nan
19806
nan
19811
nan
20234
nan
20330
nan
20426
nan
20486
nan
20568
nan
20738
nan
20981
nan
21317
nan
21681
nan
21777
nan
nan
nan


TF-IDF VECTORIZATION

PERCEPTRON

NEURAL NETS

In [20]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
#imports
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
# Enable CUDA
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [23]:
class Dataset:
    
    def __init__(self, X, y):
        self.X=torch.as_tensor(X)
        self.y=torch.as_tensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        X_1=self.X[index]
        y_1=self.y[index]
        return X_1, y_1

In [24]:
#Training dataset split into training and validation set so we have a division of 60(training), 20(validation) and 20(testing)
from sklearn.model_selection import train_test_split

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X_train1, y_train1, test_size=0.2, random_state=0)

In [25]:
#genearting the training set, testing set and the validation set
train_set = Dataset(X_train_f, y_train_f)
valid_set = Dataset(X_val_f, y_val_f)
test_set = Dataset(X_test1, y_test1)
unlabeled_set = Dataset(X_unlabeled1, np.zeros(len(X_unlabeled1)))
#Generating the data loaders for the training set, testing set and the validation set
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_set,  batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, shuffle=False)


<ipython-input-23-deb5d617ec55>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  self.X=torch.as_tensor(X)


In [26]:
# for x,y in unlabeled_loader:
#   print(x,y)
#   print(len(x))
#   break

In [27]:
#Generating the network architechture
class Net(nn.Module):
    def __init__(self, input_dim=300, output_dim=1, hidden_1=100, hidden_2=10):
        super(Net, self).__init__()
        self.input_dim = input_dim 
        self.output_dim = output_dim
        self.hidden_1 = hidden_1
        self.hidden_2 = hidden_2
        self.fc1 = nn.Linear(self.input_dim, self.hidden_1)
        self.fc2 = nn.Linear(self.hidden_1, self.hidden_2)
        self.fc3 = nn.Linear(self.hidden_2, self.output_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [28]:
#Calling the Net is_depression to initialize the neural network
model_te = Net()
print(model_te)

Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [29]:
#Defining the loss and the optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_te.parameters(), lr=0.01)

In [31]:
valid_loss_min = np.Inf
max_epochs=45
for epoch in range(max_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    model_te.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model_te(data)

        #print(output)
        #print(target)
        target = target.unsqueeze(1)
        target = target.float()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    model_te.eval()
    for data, target in valid_loader:
        output = model_te(data)
        target = target.unsqueeze(1)
        target = target.float()
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    #Printing the loss values
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_te.state_dict(), 'model_te.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.694971 	Validation Loss: 0.694970
Validation loss decreased (inf --> 0.694970).  Saving model ...
Epoch: 2 	Training Loss: 0.693746 	Validation Loss: 0.693933
Validation loss decreased (0.694970 --> 0.693933).  Saving model ...
Epoch: 3 	Training Loss: 0.693162 	Validation Loss: 0.693200
Validation loss decreased (0.693933 --> 0.693200).  Saving model ...
Epoch: 4 	Training Loss: 0.692696 	Validation Loss: 0.692830
Validation loss decreased (0.693200 --> 0.692830).  Saving model ...
Epoch: 5 	Training Loss: 0.692553 	Validation Loss: 0.692573
Validation loss decreased (0.692830 --> 0.692573).  Saving model ...
Epoch: 6 	Training Loss: 0.692436 	Validation Loss: 0.692394
Validation loss decreased (0.692573 --> 0.692394).  Saving model ...
Epoch: 7 	Training Loss: 0.692277 	Validation Loss: 0.692131
Validation loss decreased (0.692394 --> 0.692131).  Saving model ...
Epoch: 8 	Training Loss: 0.692126 	Validation Loss: 0.691906
Validation loss decreased (0.69213

In [32]:
#Function to test the model
def predict(model, dataloader):
    prediction_list = []
    truth_list = []
    m = 0
    cnt = 0
    for i, batch in enumerate(dataloader):
        outputs = model(batch[0])
        if(outputs[0] < 0.5):
          predicted = 0
        else:
          predicted = 1
        #print(batch[1], predicted)
        if(int(batch[1].item()) == int(predicted)):
          m+=1
        cnt+=1
        #_, predicted = torch.max(outputs.data, 1)
        prediction_list.append(int(predicted))
        truth_list.append(int(batch[1].item()))
    #acc = m/cnt
    return prediction_list, truth_list

In [33]:
from sklearn.metrics import classification_report

#Loading the model and generating the predictions
model_te.load_state_dict(torch.load('model_te.pt'))
pred, true_val = predict(model_te, test_loader)
print(classification_report(true_val, pred))

              precision    recall  f1-score   support

           0       0.90      0.83      0.86       778
           1       0.84      0.91      0.87       767

    accuracy                           0.87      1545
   macro avg       0.87      0.87      0.87      1545
weighted avg       0.87      0.87      0.87      1545



In [40]:
#Calling the Net is_depression to initialize the neural network
model_te = Net()
print(model_te)

Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [41]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_te.parameters(), lr=0.01)

In [42]:
#traning and validating the model
valid_loss_min = np.Inf
max_epochs=45
flag = 0
while(flag == 0):
  train_set = Dataset(X_train_f, y_train_f)
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
  unlabeled_set = Dataset(X_unlabeled1, np.zeros(len(X_unlabeled1)))
  unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, shuffle=False)
  for epoch in range(max_epochs):
      train_loss = 0.0
      valid_loss = 0.0
      model_te.train()
      for data, target in train_loader:
          optimizer.zero_grad()
          output = model_te(data)

          #print(output)
          #print(target)
          target = target.unsqueeze(1)
          target = target.float()
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
          train_loss += loss.item()*data.size(0)
      model_te.eval()
      for data, target in valid_loader:
          output = model_te(data)
          target = target.unsqueeze(1)
          target = target.float()
          loss = criterion(output, target)
          valid_loss += loss.item()*data.size(0)
      train_loss = train_loss/len(train_loader.dataset)
      valid_loss = valid_loss/len(valid_loader.dataset)
      #Printing the loss values
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch+1, 
          train_loss,
          valid_loss
          ))
      if valid_loss <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min,
          valid_loss))
          torch.save(model_te.state_dict(), 'model_te.pt')
          valid_loss_min = valid_loss
  
  flag = 1
  model_te.load_state_dict(torch.load('model_te.pt'))
  k=0
  products_list = ['laptop', 'printer', 'tablet', 'desk', 'chair']

  df_unlabeled = pd.DataFrame(zip(X_unlabeled1), columns = ['text'])
  drp = []
  print(len(X_train_f),len(y_train_f), len(X_unlabeled1))
  for i ,batch in enumerate(unlabeled_loader):
    outputs = model_te(batch[0][0])
    #print(len(batch[0][0]))
    #print(outputs)
    if(outputs[0]<= 0.1 or outputs[0] > 0.9):
      flag = 0
      X_train_f.append(batch[0][0])
      if(outputs[0] < 0.5):
        y_train_f.append(0)
      else:
        y_train_f.append(1)
      drp.append(i)
    
        #X_train_f.append(batch)
  
    #print(outputs[1])
    #break
  df_unlabeled = df_unlabeled.drop(drp)
  X_unlabeled1 = df_unlabeled['text'].values.tolist()
  print(len(X_train_f),len(y_train_f), len(X_unlabeled1))
  #print(X_unlabeled1[0])
  #print(type(X_unlabeled1[0]))
  #print(type(X_unlabeled1))



Epoch: 1 	Training Loss: 0.694589 	Validation Loss: 0.692485
Validation loss decreased (inf --> 0.692485).  Saving model ...
Epoch: 2 	Training Loss: 0.693043 	Validation Loss: 0.691897
Validation loss decreased (0.692485 --> 0.691897).  Saving model ...
Epoch: 3 	Training Loss: 0.692319 	Validation Loss: 0.691673
Validation loss decreased (0.691897 --> 0.691673).  Saving model ...
Epoch: 4 	Training Loss: 0.692028 	Validation Loss: 0.691513
Validation loss decreased (0.691673 --> 0.691513).  Saving model ...
Epoch: 5 	Training Loss: 0.691717 	Validation Loss: 0.691353
Validation loss decreased (0.691513 --> 0.691353).  Saving model ...
Epoch: 6 	Training Loss: 0.691591 	Validation Loss: 0.691174
Validation loss decreased (0.691353 --> 0.691174).  Saving model ...
Epoch: 7 	Training Loss: 0.691280 	Validation Loss: 0.690942
Validation loss decreased (0.691174 --> 0.690942).  Saving model ...
Epoch: 8 	Training Loss: 0.690958 	Validation Loss: 0.690705
Validation loss decreased (0.69094

In [44]:
from sklearn.metrics import classification_report

#Loading the model and generating the predictions
model_te.load_state_dict(torch.load('model_te.pt'))
pred, true_val = predict(model_te, test_loader)
print(classification_report(true_val, pred))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91       778
           1       0.89      0.95      0.92       767

    accuracy                           0.91      1545
   macro avg       0.92      0.91      0.91      1545
weighted avg       0.92      0.91      0.91      1545

